In [161]:
from datetime import date, timedelta

from Stock import *

s = date(2020,1,1)
e = date(2021,12,20)
tesla = Stock("tsla")
tesla.load_data()
#tesla.add_data_range(s,e,stockpath='pricedata/tsla.csv')
#tesla.save_data()

# adding individual days
#tesla.add_data(s)

done loading $tsla data.


In [162]:
# Analysis
import seaborn as sns
import matplotlib.pyplot as plt
import re

df = tesla.df.dropna()
df['domain'] = df.apply(
    lambda row: re.sub('www3.|www.','',row.link.split('//')[1].split('/')[0]), axis=1)
df1 = df.sample(frac=0.5)
df2 = df.drop(df1.index)
print(len(df))
print(len(df1))
print(len(df2))

4518
2259
2259


d:\dmarq\tools\python37\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [201]:
count = {}
for p, tx in df1[['percent_change','text']].values:
    tx = re.sub('[^a-zA-Z ]',' ',tx)
    for word in tx.split():
        if word.lower() not in count:
            count[word.lower()] = [0,0]
        count[word.lower()][p < 0] += 1

In [202]:
prob = {k:list(np.array(count[k])/(sum(count[k]))) for k in count}

In [207]:
for k in sorted(prob,key=prob.get,reverse=True):
    if (prob[k][0] >= 0.70 or prob[k][0] <= 0.30) and sum(count[k]) > 75:
        print(k,prob[k],count[k])

shall [0.983957219251337, 0.016042780748663103] [184, 3]
dated [0.972972972972973, 0.02702702702702703] [108, 3]
corporation [0.9575757575757575, 0.04242424242424243] [158, 7]
proposal [0.9523809523809523, 0.047619047619047616] [100, 5]
vote [0.9051724137931034, 0.09482758620689655] [105, 11]
officers [0.9042553191489362, 0.09574468085106383] [85, 9]
stockholders [0.8867924528301887, 0.11320754716981132] [141, 18]
committee [0.8741258741258742, 0.1258741258741259] [125, 18]
directors [0.8631178707224335, 0.13688212927756654] [227, 36]
amended [0.8414634146341463, 0.15853658536585366] [69, 13]
notice [0.828125, 0.171875] [106, 22]
meeting [0.8148148148148148, 0.18518518518518517] [242, 55]
stockholder [0.7948717948717948, 0.20512820512820512] [93, 24]
compensation [0.7771084337349398, 0.22289156626506024] [129, 37]
named [0.7608695652173914, 0.2391304347826087] [70, 22]
solarcity [0.75, 0.25] [150, 50]
subject [0.7397260273972602, 0.2602739726027397] [162, 57]
series [0.7277777777777777